<a href="https://colab.research.google.com/github/GuillermoSego/TSF/blob/main/CovarianceMatrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Programa calculo de una matriz de covarianza MxN

En la versión anterior el programa (*Tarea_5_Covarianza_MxN.ipynb*) se tenía un error de direccionamiento. Las rutinas eran correctas, sin embargo cuando se calculaba la varianza, el apuntador de los datos no estaba correctamente direccionado. Después de apuntar en la dirección correcta el programa ahora es correcto de acuerdo a las pruebas que se han realizado sobre él. 

In [1]:
%%writefile Cov.c

#include <stdio.h>

#define ST 6
#define SA 15

// Esta rutina calcula el producto punto entre dos vectores
double DotProd(double x[], double y[], int n)
{
    double sum = *(x++) * *(y ++ );
    --n ;
    
    do {
        sum += *( x++ ) * *(y ++ );
    } while (--n);
    
    return sum;
}

/*
Esta rutina calcula la varianza de los grupos de datos y llena la diagonal 
principal de la matriz de covarianza
*/
void Variance(double* sum_xPtr, double* CovPtr, double* DatPtr[], int N, int M )
{
    // En estas variables se van a hacer las sumas
    double sum_x = 0, sum_x2 = 0;  
    
    int i = 0, k = 0;
    
    do {  
    
        sum_x = 0, sum_x2 = 0;    
    
            do {
                
                // Hacemos las sumas de x y x^2
                sum_x2 += *(DatPtr[k]) * *(DatPtr[k]);
                sum_x += *(DatPtr[k] ++);
                
                i ++;
            
            } while(i<SA);
    
        // Guardamos el valor de sum x en un arreglo
        *sum_xPtr = sum_x;
        sum_xPtr ++;
        
        /*
        Calculamos la varianza y directamente guardamos estos valores en la 
        matriz de covarianza
        */
        *CovPtr = (M * (sum_x2) - (sum_x) * (sum_x)) / ( M*(M-1) );
        
        // Recorremos el apuntador para tener solo los valores de la diagonal
        CovPtr  = CovPtr + (N+1);
        
        k ++;
        i = 0;
    
    } while(k < N);
    
    
}

/*
En esta rutina se calcula la covarianza entre todas las variables y se guarda 
en la matriz de covarianza
*/
void Covariance(double* sum_xPtr, double* CovPtr, double* DatPtr[], int N, int M )
{
    
    int i = 0, k = 0;
    
    do {
        
        do {
            
            /*
            Calculamos la covarianza y lo guardamos directamente en la matriz. 
            La covarianza se calcula por renglon, un ciclo es para los renglones 
            y otro para las columnas. Avanzamos de manera que permutamos en 
            cada renglon todas las variables, permutando una variable fija con 
            todas las demas. De esta manera, si en el primer renglon tenemos x,
            hacemo xy, xz, xw, xe, xd, ... Cuando avanzamos al siguiente renglon
            hacemos yz, yw, ye, yd, ... Avanzamos zw, ze, zd, ... y asi hasta 
            acabar
            */
            *CovPtr = (M * DotProd(DatPtr[i],  DatPtr[k+1], M) - 
            (*sum_xPtr)* *(sum_xPtr + (k+1)) ) / ( M*(M-1) );
            
            //printf("%lf\n", DotProd(DatPtr[i],  DatPtr[k+1], M) );
            //printf("%d\n", k);
            
            //*CovPtr = 45;
            
            /*
            Llenamos los lugares transpuestos de la matriz sumando los lugares 
            que corresponden. (k+1)(n-1)
            */
            *(CovPtr + (k+1)*(N-1)) = *CovPtr;
            
            CovPtr ++;
            
            k ++;    
        
        // Movemos los indices para estar en la diagonal superior de la matriz
        
         } while( k < (N - 1) - i );
        
        // Apuntamos en la dirección superior de la diagonal principal 
        CovPtr = CovPtr + (i+2);
        sum_xPtr ++;
        
        k = 0;
        i ++;
    } while( i < N-1);

    
}

// Esta rutina muestra una matriz de mxn
void MatrixShow(int m, int n, double v[])
{
    
     
    int i, j, k=0;
    
    for(i=0 ; i<m ; i++){
        
        for (j=0 ; j<n ; j ++){
            
            printf("%lf \t", v[k]);
            k ++;
        }
        
        printf("\n");
    }
    
    
    
}

/*
Esta rutina hace la matriz transpuesta

Nota: Rutina innecesaria, solo se necesita una linea de codigo en la rutina 
"Covariance" para hacerse
*/
void MatrixT(int m, int n, double* CovPtr)
{
    
    int i = 0, k = 0;
    
    do {
        
        do {
            
            /*
            Llenamos la matriz transpuesta sumando los lugares que corresponden
            para estar en el lugar que buscamos
            */
            *(CovPtr + (k+1)*(n-1)) = *CovPtr;
            
            CovPtr ++;
            
            k ++;    
        
        // Movemos los indices para estar en la diagonal superior de la matriz
        } while( k < (n - 1) - i );
        
        // Apuntamos en la dirección superior de la diagonal principal 
        CovPtr = CovPtr + (i+2);
        
        k = 0;
        i ++;
    } while( i < n-1);
    
}


int main()
{
    
    // Introducimos los datos en un solo vector
    double DatM[] = {
        10.0, 10.4, 9.7, 9.7, 11.7, 11.0, 8.7, 9.5, 10.1, 9.6, 10.5, 9.2, 11.3, 10.1, 8.5,
        10.7, 9.8, 10.0, 10.1, 11.5, 10.8, 8.8, 9.3, 9.4, 9.6, 10.4, 9.0, 11.6, 9.8, 9.2,
        10.7, 9.8, 10.0, 10.1, 11.5, 10.8, 8.8, 9.3, 9.4, 9.6, 10.4, 9.0, 11.6, 9.8, 9.2,
        10.0, 10.4, 9.7, 9.7, 11.7, 11.0, 8.7, 9.5, 10.1, 9.6, 10.5, 9.2, 11.3, 10.1, 8.5,
        10.7, 9.8, 10.0, 10.1, 11.5, 10.8, 8.8, 9.3, 9.4, 9.6, 10.4, 9.0, 11.6, 9.8, 9.2,
        10.0, 10.4, 9.7, 9.7, 11.7, 11.0, 8.7, 9.5, 10.1, 9.6, 10.5, 9.2, 11.3, 10.1, 8.5
    };

    // Declaramos la matriz de la Covarianza y le apuntamos
    double CovM[ST*ST];
    double* CovPtr = CovM;
    
    // Hacemos un apuntador de apuntadores para manejar los datos
    double* DatPtr[] = {DatM, DatM + SA, DatM + 2*SA, DatM + 3*SA, DatM + 4*SA, DatM + 5*SA};

    // Declaramos un arreglo para las sumas y le apuntamos     
    double sum_xM[ST];
    double* sum_xPtr = sum_xM;

    // Varianza
    Variance(sum_xPtr, CovPtr, DatPtr, ST, SA);
    
    // Nos cercioramos que estamos apuntando en la dirección correcta
    DatPtr[ST - 1] = &DatM[0];
    
    CovPtr = CovM + 1;
    sum_xPtr = sum_xM;


    // Covarianza 
    Covariance(sum_xPtr, CovPtr, DatPtr, ST, SA);
    
    // Se imprime la matriz
    printf( "La matriz de Covarianza es: \n");
    MatrixShow(ST,ST, CovM);
    
    
    return 0;
}

Writing Cov.c


In [2]:
%%shell

gcc Cov.c -o Cov
./Cov

La matriz de Covarianza es: 
0.798571 	0.734286 	0.679286 	0.734286 	0.679286 	0.679286 	
0.734286 	0.734286 	0.734286 	0.679286 	0.734286 	0.679286 	
0.679286 	0.734286 	0.734286 	0.679286 	0.798571 	0.679286 	
0.734286 	0.679286 	0.679286 	0.798571 	0.734286 	0.679286 	
0.679286 	0.734286 	0.798571 	0.734286 	0.734286 	0.679286 	
0.679286 	0.679286 	0.679286 	0.679286 	0.679286 	0.798571 	
